In [ ]:
import pandas as pd
import warnings

In [ ]:
warnings.filterwarnings('ignore')

data = 'covid_19_clean_complete.csv'
df = pd.read_csv(data, sep=',', parse_dates=['Date'])

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Recovered']    # Active cases

**Normalization**

In [ ]:
df['Country/Region'] = df['Country/Region'].replace('Mainland China', 'China')
df[['Province/State']] = df[['Province/State']].fillna('')
df[['Confirmed', 'Deaths', 'Recovered', 'Active']] = df[['Confirmed', 'Deaths', 'Recovered', 'Active']].fillna(0)

In [ ]:
df.head()

**Grouping data**

In [ ]:
df_group = df.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
df_group.sort_values(by='Confirmed', ascending=False)

In [ ]:
temp = df.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
temp = temp.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')
temp.head()

**(PLOT)** - Cases over time

In [ ]:
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.io as pio
import plotly.express as px

init_notebook_mode(connected=True)

# Cores
recovereds = '#21bf73'
deaths = '#ff2e63'
actives = '#fe9801'

import plotly.express as px
fig = px.area(temp, 
              x="Date", 
              y="Count", 
              color='Case', 
              height=600,
              title='Cases over time',
              color_discrete_sequence = [recovereds, deaths, actives])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

**(CHOROPLETH MAP)** - Cases over time

In [ ]:
import numpy as np

fig = px.choropleth(
                    df_group,         
                    title='Cases over time',                              
                    locations="Country/Region",                                    
                    locationmode='country names', 
    
                    color=np.log(df_group["Confirmed"]), 
                    
                    hover_name='Country/Region',                                   
                    hover_data=["Confirmed", "Deaths"],
                    
                    animation_frame=df_group["Date"].dt.strftime('%d-%m-%Y'),   
                    color_continuous_scale=px.colors.sequential.Magenta)           
fig.update_layout(autosize=False, width=800, height=600)                          
fig.show()

**(CHOROPLETH MAP)** - Deaths over time

In [ ]:
fig = px.choropleth(
                    df_group,   
                    title='Deaths over time',                               
                    locations="Country/Region",                                    
                    locationmode='country names', 
    
                    color=np.log(df_group["Deaths"]), 
    
                    hover_name='Country/Region',                                   
                    hover_data=["Confirmed", "Deaths"], 
    
                    animation_frame=df_group["Date"].dt.strftime('%d-%m-%Y'),  
                    color_continuous_scale=px.colors.sequential.Magenta)           
fig.update_layout(autosize=False, width=800, height=600)
fig.show()

**(PANEL)** - Last day cases

In [ ]:
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

last_day = df[df['Date'] == max(df['Date'])]
last_day.head()

In [ ]:
fig = px.treemap(last_day.sort_values(by='Confirmed', ascending=False).reset_index(drop=True), 
                 path=["Country/Region", "Province/State"],
                 values="Confirmed",
                 height=600,
                 title='Number of confirmed cases',
                 color_discrete_sequence = px.colors.qualitative.Dark2)
fig.data[0].textinfo = 'label+text+value'
fig.show()

**(PANEL)** - Last day deaths

In [ ]:
fig = px.treemap(last_day.sort_values(by='Deaths', ascending=False).reset_index(drop=True), 
                 path=["Country/Region", "Province/State"],
                 values="Deaths",
                 height=600,
                 title='Number of confirmed deaths',
                 color_discrete_sequence = px.colors.qualitative.Dark2)
fig.data[0].textinfo = 'label+text+value'
fig.show()

**(PLOT)** - Cases over time in each country

In [ ]:
fig = px.line(df_group,
              x="Date",
              y="Confirmed",
              color='Country/Region',
              height=600,
              title='Confirmed cases',
              color_discrete_sequence = px.colors.qualitative.Dark2 )
fig.show()

**(PLOT)** - Deaths over time in each country

In [ ]:
fig = px.line(df_group,
              x="Date",
              y="Deaths",
              color='Country/Region',
              height=600,
              title='Confirmed deaths',
              color_discrete_sequence = px.colors.qualitative.Dark2)
fig.show()

**(MAP WITH FOLIUM)** - Last day cases in each county

In [ ]:
last_day = df[df['Date'] == max(df['Date'])]

import folium

world_map = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=4, zoom_start=1)

for i in range(0, len(last_day)):
    folium.Circle(
        location=[last_day.iloc[i]['Lat'], last_day.iloc[i]['Long']],
        color='crimson', fill='crimson',
        tooltip =   '<li><bold>Country : '+str(last_day.iloc[i]['Country/Region'])+
                    '<li><bold>Province : '+str(last_day.iloc[i]['Province/State'])+
                    '<li><bold>Confirmed : '+str(last_day.iloc[i]['Confirmed'])+
                    '<li><bold>Deaths : '+str(last_day.iloc[i]['Deaths']),
        radius=int(last_day.iloc[i]['Confirmed'])**1.1).add_to(world_map)
world_map